In [3]:
import torch
import torch.nn as nn
import numpy as np
import cvxpy as cp
from cvxpylayers.torch import CvxpyLayer
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing as mp
import sys
import os

# plotting
import matplotlib.pyplot as plt
import matplotlib.patheffects as patheffects

from models import *

# Problem Formulation

We solve the following MIQP problem:
$$
\begin{align}
&\text{minimize } && \mathbf{x}^T \mathbf{x} + \mathbf{p}^T \mathbf{x} \\
&\text{subject to} && \mathbf{x} \le \mathbf{b}, \\
& && \mathbf{1}^T \mathbf{x} \le a, \\
& && \mathbf{x} \le M \mathbf{y}, \\
& && \mathbf{1}^T \mathbf{y} \le 1,
\end{align}
$$
where $\mathbf{x} \in \mathbb{R}^2$ and $\mathbf{y} \in \{0,1\}^2$ are the continuous and binary optimization variables. The parameters include $\mathbf{p}$, $\mathbf{b}$, and $a$.


# Data generation

In [4]:
# Problem setup
nx = 2  # number of continuous decision variables
ny = 2  # number of integer decision variables
data_seed = 18
np.random.seed(data_seed)
torch.manual_seed(data_seed)

p_low, p_high = -30.0, 5.0   # linear term in objective
b_low, b_high = 5.0, 25.0    # RHS of constraint x <= b
a_low, a_high = 10.0, 30.0   # RHS of constraint 1^T x <= a

ntrain = 10000
ntest = 500

# Generate samples
samples_train = {
    "p": torch.FloatTensor(ntrain, nx).uniform_(p_low, p_high),
    "b": torch.FloatTensor(ntrain, nx).uniform_(b_low, b_high),
    "a": torch.FloatTensor(ntrain, 1).uniform_(a_low, a_high),
}

samples_dev = {
    "p": torch.FloatTensor(ntrain, nx).uniform_(p_low, p_high),
    "b": torch.FloatTensor(ntrain, nx).uniform_(b_low, b_high),
    "a": torch.FloatTensor(ntrain, 1).uniform_(a_low, a_high),
}

samples_test = {
    "p": torch.FloatTensor(ntest, nx).uniform_(p_low, p_high),
    "b": torch.FloatTensor(ntest, nx).uniform_(b_low, b_high),
    "a": torch.FloatTensor(ntest, 1).uniform_(a_low, a_high),
}

# --- Custom dataset class ---
class SampleDataset(torch.utils.data.Dataset):
    def __init__(self, samples):
        self.samples = samples

    def __len__(self):
        return len(self.samples["p"])

    def __getitem__(self, idx):
        return {
            "p": self.samples["p"][idx],
            "b": self.samples["b"][idx],
            "a": self.samples["a"][idx],
        }

# Create datasets
train_dataset = SampleDataset(samples_train)
dev_dataset = SampleDataset(samples_dev)
test_dataset = SampleDataset(samples_test)

# Slacked QP Formulation

We solve the following MIQP problem:
$$
\begin{align}
&\text{minimize } && \mathbf{x}^\top \mathbf{x} + \mathbf{p}^\top \mathbf{x} + \mathbf{s}^\top \mathbf{s} \\
&\text{subject to} && \mathbf{x} \le \mathbf{b}, \\
& && \mathbf{1}^\top \mathbf{x} \le a, \\
& && \mathbf{x} \le M \mathbf{y} + \mathbf{s}, \\
& && \mathbf{1}^\top \mathbf{y} \le 1, \\
& && \mathbf{s} \ge 0,
\end{align}
$$
where $\mathbf{x} \in \mathbb{R}^2$ and $\mathbf{y} \in \{0,1\}^2$ are the continuous and binary optimization variables. The parameters include $\mathbf{p}$, $\mathbf{b}$, and $a$.

In [5]:
def QP_Layer(nx, ny, penalty="l1", rho1=1.0, bigM = 1e3, **kwargs):
    # Define CVXPY variables and parameters
    x = cp.Variable((nx,))  # continuous decision variables
    y = cp.Parameter((ny,))  # integer decision variables

    p = cp.Parameter((nx,))  # linear term in the objective
    b = cp.Parameter((nx,))  # RHS of the constraint x <= b
    a = cp.Parameter((1,))   # RHS of the constraint 1.T*x <= a
    s = cp.Variable((nx,), nonneg=True)  # slack variables
    
    # Define the QP problem
    if penalty == "l1": # default to l1 penalty
        objective = cp.Minimize(cp.quad_form(x, np.eye(nx)) + p.T @ x + rho1*cp.sum(s))
    elif penalty == "l2": 
        objective = cp.Minimize(cp.quad_form(x, np.eye(nx)) + p.T @ x + rho1*cp.quad_form(s, np.eye(nx)))
    constraints = [
        x <= b,
        sum(x) <= a,
        x <= bigM * y + s,
        s >= 0,
    ]
    problem = cp.Problem(objective, constraints)

    # Create CVXPY layer
    cvxpylayer = CvxpyLayer(problem, parameters=[p, b, a, y], variables=[x, s])
    return cvxpylayer

In [6]:
@torch.no_grad()
def _check_shapes(p, b, a, y):
    """    Quick assertion helper. Accepts batched tensors:
    - p: (B,n), b: (B,n), a: (B,1) or (B,), M: (B,n,ny), y: (B,ny)
    """
    B = p.shape[0]
    n = p.shape[1]
    ny = y.shape[1]
    assert b.shape == (B, n)
    assert y.shape == (B, ny)
    assert a.shape in [(B,), (B, 1)]
    return B, n, ny

In [8]:
def solve_qp_with_slacks(layer: CvxpyLayer, p, b, a, y):
    """
    Run the CVXPYLayer in batch. All inputs are torch tensors.
    Returns x, s (each torch tensor with grad).
    """
    _, _, _ = _check_shapes(p, b, a, y)
    if a.ndim == 1:
        a = a.unsqueeze(-1)

    # Device management
    device = p.device
    layer = layer.to(device)
    p = p.to(device)
    b = b.to(device)
    a = a.to(device)
    y = y.to(device)

    x_opt, s_opt = layer(p, b, a, y)
    return x_opt, s_opt

In [9]:
def _solve_single_miqp(args):
    """Helper function to solve a single MIQP problem."""
    # Redirect stdout and stderr to devnull at the start of each process
    sys.stdout = open(os.devnull, 'w')
    sys.stderr = open(os.devnull, 'w')
    """Helper function to solve a single MIQP problem."""
    i, p_i, b_i, a_i, nx, ny = args
    
    # Variables
    x = cp.Variable(nx)
    y = cp.Variable(ny, boolean=True)
    
    # Objective and constraints
    objective = cp.Minimize(cp.sum_squares(x) + p_i @ x)
    constraints = [
        x <= b_i,
        cp.sum(x) <= a_i,
        cp.sum(y) <= 1,
        x <= 1e3 * y
    ]
    
    # Problem definition
    prob = cp.Problem(objective, constraints)
    
    try:
        prob.solve(solver=cp.GUROBI, verbose=False, OutputFlag=0)
        
        if x.value is not None and y.value is not None:
            return i, x.value, y.value
        else:
            return i, np.zeros(nx), np.zeros(ny)
    except Exception as e:
        print(f"Error solving sample {i}: {e}")
        return i, np.zeros(nx), np.zeros(ny)

@torch.no_grad()
def GUROBI_solve_parallel(p: torch.Tensor, b: torch.Tensor, a: torch.Tensor, max_workers=None):
    """
    Solve MIQP for each sample in the batch using parallel processing.
    """
    device = p.device
    p_np = p.detach().cpu().numpy()
    b_np = b.detach().cpu().numpy()
    a_np = a.detach().cpu().numpy()
    
    if a_np.ndim == 2:
        a_np = a_np.squeeze(-1)
    
    B = p_np.shape[0]
    
    # Prepare arguments for parallel execution
    args_list = [(i, p_np[i], b_np[i], a_np[i], nx, ny) for i in range(B)]
    
    # Preallocate result arrays
    x_results = np.zeros((B, nx))
    y_results = np.zeros((B, ny))
    
    # Use ProcessPoolExecutor for parallel solving
    if max_workers is None:
        max_workers = min(B, mp.cpu_count())
    
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(_solve_single_miqp, args): args[0] for args in args_list}
        
        for future in as_completed(futures):
            i, x_sol, y_sol = future.result()
            x_results[i] = x_sol
            y_results[i] = y_sol
    
    return torch.tensor(x_results).float().to(device), torch.tensor(y_results).int().to(device)

### Batch Solve Demo

In [ ]:
batch = next(iter(train_loader))
p_batch = batch['p']
b_batch = batch['b']
a_batch = batch['a']
B = p_batch.shape[0]
M_batch = torch.ones(B, nx, ny) * 50.0  # Big-M matrix

device = p_batch.device  # use your current device
B, n = p_batch.shape
ny = M_batch.shape[-1]

# 4.1 Build the layer once
cvx_layer = QP_Layer(nx=n, ny=ny, penalty="l1", rho1=1e1)

nn_model = MLPWithSTE(insize=2*nx+1, outsize=ny,
            bias=True,
            linear_map=torch.nn.Linear,
            nonlin=nn.ReLU,
            hsizes=[128] * 4)

# 4.2 Get y from your classifier
# Concatenate inputs
theta = torch.cat([p_batch, b_batch, a_batch], dim=1)  # shape: (B, nx + ny + 1)
# Forward pass through classifier
y_pred_hard = nn_model(theta).float()  # shape: (B, ny)
y_pred_int = y_pred_hard.int()

# 4.3 Solve the convex subproblem given y
x_opt, s_opt = solve_qp_with_slacks(
    cvx_layer, p_batch, b_batch, a_batch, y_pred_hard
)
obj_val = torch.sum(x_opt**2, dim=1) + torch.sum(p_batch * x_opt, dim=1)  # (B,)

# 4.4 Solve with GUROBI for comparison
x_solver, y_solver = GUROBI_solve_parallel(p_batch, b_batch, a_batch)

if True:
    # print("NN Solution x:", x_opt)
    # print("GUROBI Solution x:", x_solver)
    print("Objective values:", obj_val)
    print("Slack:", s_opt)


### Self-Supervised Learning Demo

In [ ]:
# Define some penalty functions that we may use
l1_penalty = lambda s: s.sum(dim=1)
l2_penalty = lambda s: (s**2).sum(dim=1)

def train(nn_model, cvx_layer, train_loader, slack_penalty = l1_penalty, constraint_penalty = torch.relu,
            slack_weight = 1e1, constraint_weight = 1e1,
            epochs=5, log_every=50, learning_rate=1e-4, device = None):
    
    # Put all layers in device
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
    nn_model.to(device)
    nn_model.train()
    cvx_layer.to(device)

    global_step = 0
    loss_history = []
    optimizer = torch.optim.Adam(nn_model.parameters(), lr=learning_rate)
    supervised_loss_fn = nn.BCEWithLogitsLoss() 

    for epoch in range(1, epochs+1):
        for batch in train_loader:
            p_batch = batch['p'].to(device)
            b_batch = batch['b'].to(device)
            a_batch = batch['a'].to(device)
            B = p_batch.shape[0]
            # ---- Predict y from theta = [p,b,a] ----
            theta = torch.cat([p_batch, b_batch, a_batch], dim=-1)  # (B, 2n+1)
            y_pred_hard = nn_model(theta).float() # (B, ny), hard {0,1}
            # ---- Solve convex subproblem given y ----
            # CVXPYLayer supports autograd; keep inputs requiring grad if needed
            # x_opt, s_opt = solve_qp_with_slacks(cvx_layer, p_batch, b_batch, a_batch, y_pred_hard)
            
            # May need it to include a supervised loss function 
            x_solver, y_solver = GUROBI_solve_parallel(p_batch, b_batch, a_batch)
            supervised_loss = supervised_loss_fn(y_pred_hard, y_solver.float())

            # obj_val = (x_opt**2).sum(dim=1) + (p_batch * x_opt).sum(dim=1)  # (B,)
            # # Slack penalty, for constraint violation of the continuous decision variables
            # slack_pen = slack_penalty(s_opt) 
            # # Violation penalty for constraint violation with the integer decision variables
            # y_sum_penalty = constraint_penalty(y_pred_hard.sum(dim=1) - 1.0)  # (B,)
            # # Total loss with balanced weights
            # weights = torch.tensor([1.0, slack_weight, constraint_weight, 0.0], device=device)
            # weights = weights / weights.sum()
            # loss = (
            #     weights[0]*obj_val.mean() # Main objective (no exp!)
            #     + weights[1]*slack_pen.mean() # Heavy slack penalty
            #     + weights[2]*y_sum_penalty.mean() # Integer constraint penalty
            #     # + weights[3]*y_gt_penalty.mean() # Binary sharpness regularizer
            # )

            loss = supervised_loss
            loss_history.append(loss.item())

            # ---- Backprop ----
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(nn_model.parameters(), max_norm=1e1)
            optimizer.step()

            # ---- Logging ----
            if (global_step % log_every) == 0:
                # print(f"[epoch {epoch} | step {global_step}] "
                #     f"loss={loss.item():.4f}  obj={obj_val.mean().item():.4f}  "
                #     f"slack={slack_pen.mean().item():.4f}  sum(y)={y_sum_penalty.mean().item():.3f}")
                print(f"[epoch {epoch} | step {global_step}] "
                    f"loss={loss.item():.4f}")
            global_step += 1

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
nn_model = MLPWithSTE(insize=2*nx+1, outsize=ny,
                bias=True,
                linear_map=torch.nn.Linear,
                nonlin=nn.ReLU,
                hsizes=[128] * 4)
slack_weight = 1e1
constraint_weight = 1e3
cvx_layer = QP_Layer(nx=nx, ny=ny, penalty="l1", rho1=slack_weight)

train(nn_model, cvx_layer, train_loader, 
    slack_weight = slack_weight, constraint_weight = constraint_weight,
    epochs=50, log_every=50, learning_rate=1e-3, device=device)

### Supervised learning demo

In [10]:
def train(nn_model, train_loader, test_loader, training_params, verbose=True, device = None):
    TRAINING_EPOCHS = training_params['TRAINING_EPOCHS']
    CHECKPOINT_AFTER = training_params['CHECKPOINT_AFTER']
    LEARNING_RATE = training_params['LEARNING_RATE']
    WEIGHT_DECAY = training_params['WEIGHT_DECAY']
    PATIENCE = training_params['PATIENCE']
    # Put all layers in device
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    nn_model.to(device)

    global_step = 0
    loss_history = []
    optimizer = torch.optim.Adam(nn_model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    supervised_loss_fn = nn.HuberLoss() 
    best_val_loss = float("inf") # Store best validation 
    epochs_no_improve = 0  # Count epochs with no improvement       

    for epoch in range(1, TRAINING_EPOCHS+1):
        nn_model.train()
        running_loss = 0.0        
        for batch in train_loader:
            p_batch = batch['p'].to(device)
            b_batch = batch['b'].to(device)
            a_batch = batch['a'].to(device)
            B = p_batch.shape[0]
            # ---- Predict y from theta = [p,b,a] ----
            theta = torch.cat([p_batch, b_batch, a_batch], dim=-1)  # (B, 2n+1)
            y_pred = nn_model(theta).float() # (B, ny), hard {0,1}
            
            # May need it to include a supervised loss function 
            x_solver, y_solver = GUROBI_solve_parallel(p_batch, b_batch, a_batch)

            supervised_loss = supervised_loss_fn(y_pred, y_solver.float())

            loss = supervised_loss

            # ---- Backprop ----
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(nn_model.parameters(), max_norm=1e1)
            optimizer.step()
            loss_history.append(loss.item())
        
            # ---- Logging ----
            if (global_step % CHECKPOINT_AFTER) == 0:
                training_loss = loss.item()
                val_loss_total = 0.0
                with torch.no_grad():
                    for val_batch in test_loader:
                        p_batch = val_batch['p'].to(device)
                        b_batch = val_batch['b'].to(device)
                        a_batch = val_batch['a'].to(device)
                        # ---- Predict y from theta = [p,b,a] ----
                        theta = torch.cat([p_batch, b_batch, a_batch], dim=-1)  # (B, 2n+1)
                        y_pred_test = nn_model(theta).float() # (B, ny), hard {0,1}
                        _, y_solver_test = GUROBI_solve_parallel(p_batch, b_batch, a_batch)
                        val_loss_total += supervised_loss_fn(y_pred_test, y_solver_test).item()                           
                    avg_val_loss = val_loss_total / len(test_loader)

                print(f"[epoch {epoch} | step {global_step}] "
                    f"training loss = {training_loss:.4f}, "
                    f"validation loss = {avg_val_loss:.4f}")
                
                # Check if need to update the learning rates
                last_lr = optimizer.param_groups[0]['lr']
                scheduler.step(loss.item())
                current_lr = optimizer.param_groups[0]['lr']
                if current_lr != last_lr:
                    print(f"Learning rate updated: {last_lr:.6f} -> {current_lr:.6f}")
                    last_lr = current_lr

                if avg_val_loss < best_val_loss:
                    best_val_loss = avg_val_loss
                    epochs_no_improve = 0
                else:
                    epochs_no_improve += 1
                    if epochs_no_improve >= PATIENCE:
                        print("Early stopping triggered!")
                        break
                    
            global_step += 1

In [ ]:
# Create DataLoaders
batch_size = 128
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
nn_model = MLPWithSTE(insize=2*nx+1, outsize=ny,
                bias=True,
                linear_map=torch.nn.Linear,
                nonlin=nn.ReLU,
                hsizes=[128] * 2)

training_params = {}
training_params['TRAINING_EPOCHS'] = int(50)
training_params['CHECKPOINT_AFTER'] = int(10)
training_params['LEARNING_RATE'] = 1e-3
training_params['WEIGHT_DECAY'] = 1e-5
training_params['PATIENCE'] = 10

train(nn_model, train_loader, test_loader, training_params, device=device)

[epoch 1 | step 0] training loss = 0.3281, validation loss = 0.2530
[epoch 1 | step 10] training loss = 0.1328, validation loss = 0.1664
[epoch 1 | step 20] training loss = 0.1562, validation loss = 0.1213
[epoch 1 | step 30] training loss = 0.0781, validation loss = 0.0816
[epoch 1 | step 40] training loss = 0.0703, validation loss = 0.0589
[epoch 1 | step 50] training loss = 0.0547, validation loss = 0.0528
[epoch 1 | step 60] training loss = 0.0469, validation loss = 0.0553
[epoch 1 | step 70] training loss = 0.0469, validation loss = 0.0462
[epoch 1 | step 80] training loss = 0.0312, validation loss = 0.0428
[epoch 1 | step 90] training loss = 0.0547, validation loss = 0.0405
[epoch 1 | step 100] training loss = 0.0156, validation loss = 0.0398
[epoch 1 | step 110] training loss = 0.0312, validation loss = 0.0396
[epoch 1 | step 120] training loss = 0.0312, validation loss = 0.0315
[epoch 1 | step 130] training loss = 0.0156, validation loss = 0.0309


Exception ignored in: <function _releaseLock at 0x743bb883cfe0>
Traceback (most recent call last):
  File "/home/ubuntu22/.pyenv/versions/3.12.11/lib/python3.12/logging/__init__.py", line 243, in _releaseLock
    def _releaseLock():
    
KeyboardInterrupt: 


[epoch 1 | step 140] training loss = 0.0078, validation loss = 0.0306
